In [ ]:
import torch
import pickle
model = pickle.load(open("my_model.pth","rb"),encoding="utf-8")
print("모델이 성공적으로 로드되었습니다!")

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
robot = Robot()

device = torch.device('cuda')

camera = Camera.instance(width=224, height=224,fps=1)
image = widgets.Image(format='jpeg', width=224, height=224)
stop_slider = widgets.FloatSlider(description='stop', min=0.0, max=1.0, orientation='vertical')
green_slider = widgets.FloatSlider(description='green', min=0.0, max=1.0, orientation='vertical')
orange_slider = widgets.FloatSlider(description='orange', min=0.0, max=1.0, orientation='vertical')
dislike_slider = widgets.FloatSlider(description='dislike', min=0.0, max=1.0, orientation='vertical')


camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(widgets.HBox([image, dislike_slider,orange_slider,green_slider,stop_slider]))

In [ ]:
import cv2
import numpy as np
import torch
import torchvision

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.cuda()
    x = x[None, ...]
    return x

In [ ]:
Speed = 0.7
L_F_MPR = 1.00
L_B_MPR =1.00
R_F_MPR= 1.00
R_B_MPR = 1.00

def Move2(speedLeft, speedRight):
    L_F_MPR = 1.00
    L_B_MPR =1.00
    R_F_MPR= 1.00
    R_B_MPR = 1.00
    if speedLeft > 0:
        robot.left_motor.value = speedLeft * L_F_MPR
    else:
        robot.left_motor.value = speedLeft * L_B_MPR
    if speedRight > 0:
        robot.right_motor.value = speedRight * R_F_MPR
    else:
        robot.right_motor.value = speedRight * R_B_MPR

def Move(speed):
    Move2(speed, speed)

def Stop():
    robot.left_motor.value = 0
    robot.right_motor.value = 0


In [ ]:
import os
os.system("sudo chmod 777 /dev/ttyTHS1")

from servoserial import ServoSerial

In [ ]:
import torch.nn.functional as F
import time


def update(change):
    global robot
    x = change['new']
    x = preprocess(x)
    print(x)
    y = model((x))
    print(y)
    
    y = F.softmax(y,dim=1)
    
    prob_stop = float(y.flatten()[3])
    prob_orange = float(y.flatten()[1])
    prob_green = float(y.flatten()[0])
    prob_dislike = float(y.flatten()[2])
    
    stop_slider.value = prob_stop
    green_slider.value = prob_green
    orange_slider.value = prob_orange 
    dislike_slider.value = prob_dislike 
    
    if prob_stop > 0.6:
        robot.stop()
        
    elif prob_green > 0.6:      
        robot.forward(0.8)
        time.sleep(0.5)
        robot.stop()
        
    elif prob_orange >0.6:  
        robot.backward(0.8)
        time.sleep(0.5)
        robot.stop()
        
    elif prob_dislike >0.6:
        for i in range(0,repeat=3):
            pantilt.Servo_serial_control(1, MID_PAN+200)
            Move2(-speed, speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN+400)
            Move2(-speed, speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN+200)
            Stop()
            time.sleep(delay)

            pantilt.Servo_serial_control(1, MID_PAN-200)
            Move2(speed, -speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN-400)
            Move2(speed, -speed)
            time.sleep(delay)
            pantilt.Servo_serial_control(1, MID_PAN-200)
            Stop()
            time.sleep(delay)

            pantilt.Servo_serial_control(1, MID_PAN)
            Stop()
            time.sleep(delay)
        
update({'new': camera.value})
camera.observe(update,names = 'value')  